In [ ]:
#-*- coding: utf-8 -*-
#한글쓰기위에서 위 코드 추가
import numpy as np
import tensorflow as tf

In [ ]:
training_epochs = 500 # 학습 횟수
n_dim = 193
n_classes = 3
learning_rate = 0.01 # 학습 비율
sd = 1/np.sqrt(n_dim) # 표준 편차

In [3]:
n_hidden_units_one = 300
n_hidden_units_two = 200
n_hidden_units_three = 100

In [4]:
X = tf.placeholder(tf.float32,[None,n_dim]) # 입력 데이터 파라미터
Y = tf.placeholder(tf.float32,[None,n_classes]) # 정답 데이터 파라미터

In [5]:
# 1차 히든 레이어(원소까지 랜덤인 배열 생성)
W_1 = tf.Variable(tf.random_normal([n_dim, n_hidden_units_one], mean=0, stddev=sd), name="w1")
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean=0, stddev=sd), name="b1")
h_1 = tf.nn.relu(tf.matmul(X, W_1) + b_1)

In [6]:
# 2차 히든 레이어
W_2 = tf.Variable(tf.random_normal([n_hidden_units_one, n_hidden_units_two], mean=0, stddev=sd), name="w2")
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean=0, stddev=sd), name="b2")
h_2 = tf.nn.tanh(tf.matmul(h_1, W_2) + b_2)

In [7]:
# 3차 히든 레이어
W_3 = tf.Variable(tf.random_normal([n_hidden_units_two, n_hidden_units_three], mean=0, stddev=sd), name="w3")
b_3 = tf.Variable(tf.random_normal([n_hidden_units_three], mean=0, stddev=sd), name="b3")
h_3 = tf.nn.relu(tf.matmul(h_2, W_3) + b_3)

In [8]:
# 드롭아웃 과정 추가
keep_prob = tf.placeholder(tf.float32)
h_3_drop = tf.nn.dropout(h_3, keep_prob, name="h3")

In [9]:
W = tf.Variable(tf.random_normal([n_hidden_units_three, n_classes], mean=0, stddev=sd), name="w")
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd), name="b")
y_ = tf.nn.softmax(tf.matmul(h_3_drop, W) + b, name="op_to_restore")

cross_entropy = -tf.reduce_sum(Y*tf.log(tf.clip_by_value(y_, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


In [14]:
saver = tf.train.Saver()


correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    sound_data = np.load('workspace/sound_data.npz')
    test_data = np.load('workspace/dog_data.npz')
    X_data = sound_data['X']
    y_data = sound_data['y']
    X_test = test_data['X']
    
    for epoch in range(training_epochs):
        sess.run(train_step, feed_dict={X: X_data, Y: y_data, keep_prob: 0.5 })
        
        if epoch%100 == 0 :
            train_accuracy = sess.run(accuracy, feed_dict={X: X_data, Y: y_data, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(epoch, train_accuracy))
            
    save_path = saver.save(sess, "workspace/sound_model.ckpt")
    
    answer = sess.run(y_, feed_dict={X: X_test, keep_prob: 0.5})
    index = tf.argmax(answer,1).eval(session=sess)
    
    print(answer)
    print(index)
    
    raw = answer[0,index]*100
    
    print(raw)
    
    if(10>raw) :
        print("일치하는 소리가 없습니다.")
    else :
        print("분류된 소리는 : %.2f 로 뭐입니다" % raw)
    
    
    

    
    
    
    

step 0, training accuracy 0.35
step 100, training accuracy 1
step 200, training accuracy 1
step 300, training accuracy 1
step 400, training accuracy 1
[[4.4291466e-03 9.9479854e-01 7.7230402e-04]]
[1]
[99.47985]
분류된 소리는 : 99.48 로 뭐입니다
